# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 27 2022 8:30AM,249465,10,MSP215263,"Methapharm, Inc.",Released
1,Oct 27 2022 8:30AM,249465,10,MSP215265,"Methapharm, Inc.",Executing
2,Oct 27 2022 8:30AM,249465,10,MSP215267,"Methapharm, Inc.",Executing
3,Oct 27 2022 8:30AM,249465,10,MSP215268,"Methapharm, Inc.",Executing
4,Oct 27 2022 8:30AM,249465,10,MSP215273,"Methapharm, Inc.",Executing
5,Oct 27 2022 8:30AM,249465,10,MSP215277,"Methapharm, Inc.",Executing
6,Oct 26 2022 3:54PM,249466,16,8692968,Sartorius Bioprocess Solutions,Released
7,Oct 26 2022 3:28PM,249462,10,0086119092,ISDIN Corporation,Released
8,Oct 26 2022 3:28PM,249462,10,0086119093,ISDIN Corporation,Released
9,Oct 26 2022 3:28PM,249462,10,0086119107,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
12,249461,Released,1
13,249462,Released,5
14,249465,Executing,5
15,249465,Released,1
16,249466,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
249460,NaN,1.0
249461,NaN,1.0
249462,NaN,5.0
249465,5.0,1.0
249466,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
249407,0.0,1.0
249408,0.0,1.0
249418,0.0,1.0
249447,0.0,1.0
249451,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
249407,0,1
249408,0,1
249418,0,1
249447,0,1
249451,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,249407,0,1
1,249408,0,1
2,249418,0,1
3,249447,0,1
4,249451,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,249407,,1
1,249408,,1
2,249418,,1
3,249447,,1
4,249451,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 27 2022 8:30AM,249465,10,"Methapharm, Inc."
6,Oct 26 2022 3:54PM,249466,16,Sartorius Bioprocess Solutions
7,Oct 26 2022 3:28PM,249462,10,ISDIN Corporation
12,Oct 26 2022 3:22PM,249455,10,ISDIN Corporation
28,Oct 26 2022 3:08PM,249461,22,"NBTY Global, Inc."
29,Oct 26 2022 3:07PM,249460,22,"NBTY Global, Inc."
30,Oct 26 2022 3:04PM,249459,12,"SD Head USA, LLC"
31,Oct 26 2022 2:40PM,249457,10,ISDIN Corporation
47,Oct 26 2022 2:10PM,249454,10,ISDIN Corporation
50,Oct 26 2022 1:51PM,249453,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Oct 27 2022 8:30AM,249465,10,"Methapharm, Inc.",5,1
1,Oct 26 2022 3:54PM,249466,16,Sartorius Bioprocess Solutions,,1
2,Oct 26 2022 3:28PM,249462,10,ISDIN Corporation,,5
3,Oct 26 2022 3:22PM,249455,10,ISDIN Corporation,,16
4,Oct 26 2022 3:08PM,249461,22,"NBTY Global, Inc.",,1
5,Oct 26 2022 3:07PM,249460,22,"NBTY Global, Inc.",,1
6,Oct 26 2022 3:04PM,249459,12,"SD Head USA, LLC",,1
7,Oct 26 2022 2:40PM,249457,10,ISDIN Corporation,,16
8,Oct 26 2022 2:10PM,249454,10,ISDIN Corporation,2,1
9,Oct 26 2022 1:51PM,249453,10,ISDIN Corporation,,8


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 27 2022 8:30AM,249465,10,"Methapharm, Inc.",1,5
1,Oct 26 2022 3:54PM,249466,16,Sartorius Bioprocess Solutions,1,
2,Oct 26 2022 3:28PM,249462,10,ISDIN Corporation,5,
3,Oct 26 2022 3:22PM,249455,10,ISDIN Corporation,16,
4,Oct 26 2022 3:08PM,249461,22,"NBTY Global, Inc.",1,
5,Oct 26 2022 3:07PM,249460,22,"NBTY Global, Inc.",1,
6,Oct 26 2022 3:04PM,249459,12,"SD Head USA, LLC",1,
7,Oct 26 2022 2:40PM,249457,10,ISDIN Corporation,16,
8,Oct 26 2022 2:10PM,249454,10,ISDIN Corporation,1,2
9,Oct 26 2022 1:51PM,249453,10,ISDIN Corporation,8,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 27 2022 8:30AM,249465,10,"Methapharm, Inc.",1,5
1,Oct 26 2022 3:54PM,249466,16,Sartorius Bioprocess Solutions,1,
2,Oct 26 2022 3:28PM,249462,10,ISDIN Corporation,5,
3,Oct 26 2022 3:22PM,249455,10,ISDIN Corporation,16,
4,Oct 26 2022 3:08PM,249461,22,"NBTY Global, Inc.",1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Oct 27 2022 8:30AM,249465,10,"Methapharm, Inc.",1,5.0
1,Oct 26 2022 3:54PM,249466,16,Sartorius Bioprocess Solutions,1,NaN
2,Oct 26 2022 3:28PM,249462,10,ISDIN Corporation,5,NaN
3,Oct 26 2022 3:22PM,249455,10,ISDIN Corporation,16,NaN
4,Oct 26 2022 3:08PM,249461,22,"NBTY Global, Inc.",1,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 27 2022 8:30AM,249465,10,"Methapharm, Inc.",1,5.0
1,Oct 26 2022 3:54PM,249466,16,Sartorius Bioprocess Solutions,1,0.0
2,Oct 26 2022 3:28PM,249462,10,ISDIN Corporation,5,0.0
3,Oct 26 2022 3:22PM,249455,10,ISDIN Corporation,16,0.0
4,Oct 26 2022 3:08PM,249461,22,"NBTY Global, Inc.",1,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,1496746,47,7.0
12,498877,2,0.0
16,249466,1,0.0
19,249447,1,0.0
21,498858,2,0.0
22,748329,3,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,1496746,47,7.0
1,12,498877,2,0.0
2,16,249466,1,0.0
3,19,249447,1,0.0
4,21,498858,2,0.0
5,22,748329,3,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,47,7.0
1,12,2,0.0
2,16,1,0.0
3,19,1,0.0
4,21,2,0.0
5,22,3,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,47.0
1,12,Released,2.0
2,16,Released,1.0
3,19,Released,1.0
4,21,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,19,21,22
Status,,,,,,
Executing,7.0,0.0,0.0,0.0,0.0,0.0
Released,47.0,2.0,1.0,1.0,2.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,19,21,22
0,Executing,7.0,0.0,0.0,0.0,0.0,0.0
1,Released,47.0,2.0,1.0,1.0,2.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,19,21,22
0,Executing,7.0,0.0,0.0,0.0,0.0,0.0
1,Released,47.0,2.0,1.0,1.0,2.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()